# Create a virtual environment on TSCC

Virtual environments are isolated programming environments that allow users to install packages specific to individual projects, without worrying about compatability among dependency issues. For example, if you were working on **Project A** which requires Package **V1.0**, but also working on **Project B** which requires Package **V2.0**, you could create two environments and work on both without worrying about any version conflicts. We are going to be working mostly in python3 for this course, but sometimes we still need python2. So we are going to create two different envrionments which will allow you to work in either version as needed.

## 1) Make python2 virtual environment

There are a few ways to create virtual environments, but the easiest way for us to just use conda:
```bash
conda create --name py2_env python=2.7 numpy pandas scipy scikit-learn matplotlib seaborn jupyter```

which creates a conda environment called py2_env, and with the following packages installed:
- **python** (version 2.7)
- **numpy, pandas, scipy, scikit-learn, matplotlib, seaborn, jupyter** (any versions that are compatible with each other). These are the most common packages you will use, but you can add more later. 

Check if it worked! You can list all environments that you have created with the following command: 
```bash
conda env list```

Go into that environment with:
```bash
source activate py2_env```

Notice that you now have a parenthesis on the command line before your username that tells you what environment you are working in. 


While we are in here, we are going to add this environment as a kernel in jupyter (more explanation later). Do that with the following command:
```bash
python -m ipykernel install --user --display-name "py2_env"```

Once that has finished get out of the environment with:
```bash
source deactivate```

And refresh your normal settings by sourcing your bashrc file: 
```bash
source ~/.bashrc```

## 2) Make python3 virtual environment

For this course, we will be primarily working in Python3 because the clustering algorithm that we want to use is written in python3. We are going to make another environment for python3 and install all the programs we will need inside of it.  
```bash
conda create --yes -n py3_cshl python=3 scipy numpy jupyter scikit-learn numpy matplotlib seaborn pandas statsmodels seaborn ipykernel```

Move into that environment with source activate:
```bash
source activate py3_cshl```

Install [phenograph](https://github.com/jacoblevine/PhenoGraph) with pip:
```bash
pip install git+https://github.com/jacoblevine/phenograph.git```

Add this environment as a kernel in jupyter:
```bash
python -m ipykernel install --user --display-name "py3_cshl"```

Get out of the environment and source your .bashrc file:
```bash
source deactivate```

```bash
source ~/.bashrc```

## 3) Make R environment for Seurat
Create another environment called "R-Seurat". This code below creates and activates our environment, and also uses 3 [channels](https://conda.io/docs/glossary.html#channels) (anaconda, r, conda-forge) to install some packages into our environment:

```bash
conda create -y -n R-Seurat 
source activate R-Seurat
conda install -y -c anaconda-forge gcc=4.8.5 
conda install -y -c anaconda jupyter=1.0.0
conda install -y -c r r-base=3.4.1
conda install -y -c conda-forge libjpeg-turbo=1.5 zeromq=4.2.5
```

### Dependency hell
While conda does its best to manage packages/dependencies, there may be bugs. Unfortunately in this case, there is an error in the specific C library that is softlinked, as well as a path to an environment variable that needs to be linked. We'll need to fix this, and to do so, run through the steps very carefully:

Set some variables in shell
```bash
ENVNAME=R-single-cell
ANACONDAENV=/home/ucsd-trainXY/anaconda2/envs # replace 'ucsd-trainXY' with your username
```

Copy and run the following block of code, which:
- Changes directory to where the problem library is
- Moves the softlink somewhere else (just in case)
- Softlinks the appropriate library files (libstdc++.so.6.0.24 -> ./libstdc++.so, libstdc++.so.6.0.24 -> ./libstdc++.so.6)

```bash
cd ${ANACONDAENV}/${ENVNAME}/lib;

mv libstdc++.so ./libstdc++.so-bak;
mv libstdc++.so.6 ./libstdc++.so.6-bak;

ln -s libstdc++.so.6.0.24 ./libstdc++.so;
ln -s libstdc++.so.6.0.24 ./libstdc++.so.6;
```

### Installing Seurat inside R

The next step is to install the proper package dependencies required by Seurat. First, however, let's make sure we're using the correct R (that was installed from the r conda channel above):
```bash
which R
```

You should see:
```bash
~/anaconda2/envs/seurat/bin/R
```

Run R inside your terminal by typing: 
```bash
R
```

You should see:
```R
R version 3.4.1 (2017-06-30) -- "Single Candle"
Copyright (C) 2017 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> 
```

First, let's install devtools, which is required by Seurat and also lets us specify specific versions of packages to install:
```R
install.packages('devtools') # Pick http and a region that's close to us
```

Next, install stringi:
```R
install.packages("stringi", configure.args=c("--disable-cxx11"), repos="https://cran.rstudio.com")
```

After installing these packages, you should be able to run this entire block of code without any problems:
```R
devtools::install_version("uuid","0.1-2")
devtools::install_version("repr","0.15.0")
devtools::install_version("IRdisplay","0.5.0")
devtools::install_version("evaluate","0.10.1")
devtools::install_version("crayon","1.3.4")
devtools::install_version("digest","0.6.15")
devtools::install_version("curl","3.2")
devtools::install_version("lubridate","1.7.4")
devtools::install_version("Rcpp","0.12.17")
devtools::install_version("httpuv", "1.3.5") # current version breaks
devtools::install_version("shiny", "1.0.5") # current shiny requires httpuv 1.4.3
devtools::install_version("pillar", "1.2.1") # current pull looks for nonexist 1.2.2
devtools::install_version("CVST", "0.2") # current pull looks for nonexist
devtools::install_version("DRR", "0.0.3")
devtools::install_version("dimRed", "0.1.0")
devtools::install_version("Hmisc", "4.1-1")
devtools::install_version("htmlTable", "1.11.1") # current pull references nonexist
devtools::install_version("recipes", "0.1.2")
devtools::install_version("tibble", "1.4.2")
devtools::install_version("ggplot2", "2.2.1")
devtools::install_version("crosstalk", "1.0.0")
devtools::install_version("plotly", "4.7.1")
devtools::install_version("caret", "6.0-78") # current pull references nonexist
devtools::install_version("Seurat","2.3.1")
devtools::install_version("pbdZMQ","0.3-3")
```

### Installing DESeq2 inside R
Sometimes, we can leverage a package manager called bioconductor to help us install packages (we would have done this for Seurat if Seurat were part of bioconductor). Fortunately, DESeq2 can be installed in this manner:

Run this inside R
```R
source("https://bioconductor.org/biocLite.R")
biocLite("DESeq2")  # upgrade all should be fine, otherwise upgrade none
```

### Making Seurat and DESeq available in your jupyter notebook
We can load and use these packages inside the terminal as-is, however it's nice to be able to view things like plots and stuff inside Jupyter. Let's run the appropriate commands inside R, which installs IRKernel, and registers our environment (single-cell) to our Jupyter installation:

```R
options(unzip = 'internal')
devtools::install_github('IRkernel/IRkernel')
IRkernel::installspec(name = 'single-cell', displayname = 'single-cell')
```